<a href="https://colab.research.google.com/github/AzlinRusnan/Optimizing_CSAT_Through_Sentiment-Analysis_and_Predictive-Modeling/blob/main/Fine_tuning_a_pre_trained_BERT_model_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn
!pip install transformers datasets torch scikit-learn pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import libraries
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [ ]:
# Load data
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from transformers import BertConfig, BertForSequenceClassification

file_path = '/content/gdrive/MyDrive/Capstone Project/fine-tuned-no-neutral.xlsx'
data = pd.read_excel(file_path)

# Map sentiment labels to integers
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
data['Label'] = data['Sentiment'].map(label_mapping)

# Drop rows with missing or duplicate comments
data = data.dropna(subset=['Comment', 'Label']).drop_duplicates(subset=['Comment'])

# Check data
print(data.head())

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Comment'].tolist(),
    data['Label'].tolist(),
    test_size=0.2,
    random_state=42
)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Define dataset class
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

model = BertForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment",
    num_labels=3,
    ignore_mismatched_sizes=True
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",  # Disables W&B
    load_best_model_at_end=True
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

                                             Comment Sentiment  Label
0  Thanks for punctual and quick service resolvin...  Positive      2
1                                    Prompt response  Positive      2
2                                      Quick support  Positive      2
3  Andre provided excellent support He used Teams...  Positive      2
4  Assistance was immediate and resolved my issue...  Positive      2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.438500,0.122873
2,0.065200,0.142782
3,0.038900,0.158334


KeyboardInterrupt: 

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
#evaluate the model on the validation set

from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Get predictions and labels
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Print classification report
print(classification_report(labels, preds, target_names=['Negative','Positive']))

# Calculate overall accuracy
accuracy = accuracy_score(labels, preds)
print(f"Validation Accuracy: {accuracy:.2f}")

              precision    recall  f1-score   support

    Negative       1.00      0.94      0.97        17
    Positive       0.95      1.00      0.98        20

    accuracy                           0.97        37
   macro avg       0.98      0.97      0.97        37
weighted avg       0.97      0.97      0.97        37

Validation Accuracy: 0.97


In [ ]:
# test the fine-tuned model

from transformers import pipeline

fine_tuned_model = BertForSequenceClassification.from_pretrained('./fine_tuned_model')
fine_tuned_tokenizer = BertTokenizer.from_pretrained('./fine_tuned_model')

# Create a sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Test with new comments
test_comments = ["Very long journey! Over months!"]
results = sentiment_analyzer(test_comments)
print(results)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.9494350552558899}]


##### **Download the fine-tuned model**

In [ ]:
import os

# Check if the folder exists
print(os.path.exists('./fine_tuned_model'))

# List files in the folder
print(os.listdir('./fine_tuned_model'))

True
['model.safetensors', 'config.json', 'vocab.txt', 'tokenizer_config.json', 'special_tokens_map.json']


In [ ]:
!zip -r fine_tuned_model.zip ./fine_tuned_model

  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/model.safetensors (deflated 7%)
  adding: fine_tuned_model/config.json (deflated 54%)
  adding: fine_tuned_model/vocab.txt (deflated 48%)
  adding: fine_tuned_model/tokenizer_config.json (deflated 76%)
  adding: fine_tuned_model/special_tokens_map.json (deflated 42%)


In [ ]:
from google.colab import files
files.download('fine_tuned_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### **Test on Real Dataset**

In [ ]:
# Load the real dataset for testing
test_file_path = '/content/gdrive/MyDrive/Capstone Project/Sentiment Analysis_cleaned_dataset_Aug.xlsx'
test_data = pd.read_excel(test_file_path)

# Ensure the dataset has a column for comments
test_comments = test_data['USS Comment'].dropna().tolist()

# Perform predictions on the test dataset
results = sentiment_analyzer(test_comments)

# Add results to the original dataset
test_data['Predicted Sentiment'] = [res['label'] for res in results]
test_data['Confidence'] = [res['score'] for res in results]

test_data.to_excel('CSAT_SAAUG1_DATASET.xlsx', index=False)

from google.colab import files
files.download('CSAT_SAAUG1_DATASET.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>